In [1]:
from functionUtils import *
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm_notebook, tnrange
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import datetime
import gc
DATA_PATH = './datasets/'
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
usecols=['card_id','merchant_id','merchant_category_id','purchase_amount','month_lag','month_gap']
df_transactions = pd.read_csv('./datasets/df_transactions.csv',usecols=usecols)
df_historical_transactions = df_transactions[df_transactions.month_lag<=0]
df_new_transactions = df_transactions[df_transactions.month_lag>0]

df_merchants = pd.read_csv(DATA_PATH+'merchants.csv')
df_merchants.drop_duplicates('merchant_id',inplace=True)
# df_merchants.sort_values(by=['merchant_category_id'],ascending=True,inplace=True)
df_merchants['category_2'].fillna(6,inplace=True)
df_merchants = reduce_mem_usage(df_merchants)

Memory usage of dataframe is 58.72 MB
Memory usage after optimization is: 32.87 MB
Decreased by 44.0%


In [3]:
df_merchants['avg_sales_lag3'].fillna(df_merchants['avg_sales_lag3'].mean(),inplace=True)
df_merchants['avg_sales_lag6'].fillna(df_merchants['avg_sales_lag6'].mean(),inplace=True)
df_merchants['avg_sales_lag12'].fillna(df_merchants['avg_sales_lag12'].mean(),inplace=True)

In [4]:
df_merchants['avg_purchases_lag6'].replace([np.inf,-np.inf],np.nan,inplace=True)
df_merchants['avg_purchases_lag12'].replace([np.inf,-np.inf],np.nan,inplace=True)
df_merchants['avg_purchases_lag6'].fillna(df_merchants['avg_purchases_lag6'].median(),inplace=True)
df_merchants['avg_purchases_lag12'].fillna(df_merchants['avg_purchases_lag12'].median(),inplace=True)

In [5]:
df_merchants.head(n=100)

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057465,-0.057465,N,E,E,-0.400000,...,-2.250000e+00,18.666667,6,-2.320000e+00,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057465,-0.057465,N,E,E,-0.720000,...,-7.400000e-01,1.291667,6,-5.700000e-01,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057465,-0.057465,N,E,E,-82.129997,...,-8.213000e+01,260.000000,2,-8.213000e+01,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057465,-0.057465,Y,E,E,13.840252,...,2.165616e+01,4.666667,6,2.523281e+01,3.833333,12,Y,-1,-1,1.0
4,M_ID_64456c37ce,2228,222,21,-0.057465,-0.057465,Y,E,E,13.840252,...,2.165616e+01,0.361111,6,2.523281e+01,0.347222,12,Y,-1,-1,1.0
5,M_ID_a0915f62b5,20201,87,27,-0.057465,-0.057465,N,E,E,13.840252,...,2.165616e+01,3.666667,6,2.523281e+01,3.833333,12,Y,160,21,5.0
6,M_ID_bfd41933db,33861,792,9,-0.057465,-0.057465,N,E,E,13.840252,...,2.165616e+01,4.833333,6,2.523281e+01,6.333333,12,N,60,16,1.0
7,M_ID_d8ff08219e,16430,529,20,-0.057465,-0.057465,Y,E,E,13.840252,...,2.165616e+01,1.666667,6,2.523281e+01,1.500000,11,Y,-1,-1,1.0
8,M_ID_c5b389236d,37179,813,29,-0.057465,-0.057465,N,E,E,13.840252,...,2.165616e+01,189.916667,6,2.523281e+01,197.000000,7,N,248,15,1.0
9,M_ID_d2162ed113,112122,81,29,-0.057465,-0.057465,Y,E,E,13.840252,...,2.165616e+01,1.000000,2,2.523281e+01,1.000000,2,Y,-1,-1,1.0


In [6]:
df_merchants['numerical_remain'] = df_merchants['numerical_1'] - df_merchants['numerical_2']
df_merchants['avg_purchases_ratio_3/6'] = (df_merchants['avg_purchases_lag6'] - df_merchants['avg_purchases_lag3'])/df_merchants['avg_purchases_lag3']
df_merchants['avg_purchases_ratio_6/12'] = (df_merchants['avg_purchases_lag12'] - df_merchants['avg_purchases_lag6'])/df_merchants['avg_purchases_lag6']
for index in [3,6,12]:
    df_merchants['avg_remain_lag%s'%index] = df_merchants['avg_purchases_lag%s'%index] - df_merchants['avg_sales_lag%s'%index]

#商家出现的频次和频率特征
df_temp = df_historical_transactions.merchant_id.value_counts().reset_index()
df_temp.rename(columns={'index':'merchant_id','merchant_id':'hist_merchant_id_counts'},inplace=True)
df_temp['hist_merchant_id_ratio'] = df_temp['hist_merchant_id_counts']/df_historical_transactions.shape[0]
df_merchants = df_merchants.merge(df_temp,on='merchant_id',how='left')

df_temp = df_new_transactions.merchant_id.value_counts().reset_index()
df_temp.rename(columns={'index':'merchant_id','merchant_id':'new_merchant_id_counts'},inplace=True)
df_temp['new_merchant_id_ratio'] = df_temp['new_merchant_id_counts']/df_new_transactions.shape[0]
df_merchants = df_merchants.merge(df_temp,on='merchant_id',how='left')
df_merchants[['hist_merchant_id_counts','hist_merchant_id_ratio','new_merchant_id_counts','new_merchant_id_ratio']].fillna(0,inplace=True)

In [9]:
df_merchants.head()

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,numerical_remain,avg_purchases_ratio_3/6,avg_purchases_ratio_6/12,avg_remain_lag3,avg_remain_lag6,avg_remain_lag12,hist_merchant_id_counts,hist_merchant_id_ratio,new_merchant_id_counts,new_merchant_id_ratio
0,M_ID_838061e48c,8353,792,9,-0.057465,-0.057465,N,E,E,-0.400000,...,0.0,0.931034,-0.254464,10.066667,20.916667,16.236667,6.0,2.060980e-07,NaN,NaN
1,M_ID_9339d880ad,3184,840,20,-0.057465,-0.057465,N,E,E,-0.720000,...,0.0,-0.261905,0.306452,2.470000,2.031667,2.257500,3.0,1.030490e-07,NaN,NaN
2,M_ID_e726bbae1e,447,690,1,-0.057465,-0.057465,N,E,E,-82.129997,...,0.0,0.000000,0.000000,342.129997,342.129997,342.129997,12.0,4.121960e-07,NaN,NaN
3,M_ID_a70e9c5f81,5026,792,9,-0.057465,-0.057465,Y,E,E,13.840252,...,0.0,1.800000,-0.178571,-12.173585,-16.989490,-21.399480,7.0,2.404477e-07,1.0,5.094163e-07
4,M_ID_64456c37ce,2228,222,21,-0.057465,-0.057465,Y,E,E,13.840252,...,0.0,-0.277778,-0.038462,-13.340252,-21.295045,-24.885591,2.0,6.869934e-08,NaN,NaN


In [10]:
#商家统计特征
for col,df_feature in zip(['hist','new'],[df_historical_transactions,df_new_transactions]):
    print('.'*20+'%s'%col+'.'*20)
    df_merchants = getMeanStaticsFeatures(df_merchants,df_feature,group='merchant_id',fea='purchase_amount',name='%s_merchantPurchaseMean'%col)
    df_merchants = getMaxStaticsFeatures(df_merchants,df_feature,group='merchant_id',fea='purchase_amount',name='%s_merchantPurchaseMax'%col)
    df_merchants = getStdStaticsFeatures(df_merchants,df_feature,group='merchant_id',fea='purchase_amount',name='%s_merchantPurchaseStd'%col)
    df_merchants = getSumStaticsFeatures(df_merchants,df_feature,group='merchant_id',fea='purchase_amount',name='%s_merchantPurchaseSum'%col)
    
    df_merchants = getMeanStaticsFeatures(df_merchants,df_feature,group='merchant_category_id',fea='purchase_amount',name='%s_merchaCidPurchaseMean'%col)
    df_merchants = getMaxStaticsFeatures(df_merchants,df_feature,group='merchant_category_id',fea='purchase_amount',name='%s_merchaCidPurchaseMax'%col)
    df_merchants = getStdStaticsFeatures(df_merchants,df_feature,group='merchant_category_id',fea='purchase_amount',name='%s_merchaCidPurchaseStd'%col)
    df_merchants = getSumStaticsFeatures(df_merchants,df_feature,group='merchant_category_id',fea='purchase_amount',name='%s_merchaCidPurchaseSum'%col)

#商家月销售特征
def getMonthSales(df_data,df_feature,group='month_gap',fea='purchase_amount',name='hist'):
    df_purchase = df_feature.groupby(['merchant_id','month_gap'])['purchase_amount'].sum().reset_index()
    df_purchase.rename(columns={'purchase_amount':'purchase_amount_sum_month'},inplace=True)
    df_purchase.sort_values(by=['month_gap'],inplace=True,ascending=True)
    df_count = df_purchase.groupby(['merchant_id'])['purchase_amount_sum_month'].count().reset_index()
    df_count.rename(columns={'purchase_amount_sum_month':'%s_purchase_counts'%name},inplace=True)

    df_temp = df_purchase[['merchant_id','month_gap','purchase_amount_sum_month']]
    df_temp.index = df_temp.merchant_id
    df_temp = df_temp.set_index(['month_gap'],append=True)
    df_temp = pd.Series(df_temp['purchase_amount_sum_month'].values.reshape(len(df_temp['purchase_amount_sum_month'])),index=df_temp.index)
    df_temp = df_temp.unstack()
    df_temp.reset_index(inplace=True)
    cols = ['merchant_id']
    for index in list(df_purchase['month_gap'].unique()):
        cols.append('%s_month%s_purchase'%(name,index))
    df_temp.columns = cols
    df_temp = df_temp.merge(df_count,on='merchant_id',how='left')
    df_temp.fillna(0,inplace=True)
    df_temp['%s_purchase_mean'%name] = 0
    for index in list(df_purchase['month_gap'].unique()):
        df_temp['%s_purchase_mean'%name] +=df_temp['%s_month%s_purchase'%(name,index)]
    if name=='newMerch':
        df_temp['%s_purchase_mean'%name] = (df_temp['%s_purchase_mean'%name] - df_temp['%s_month1_purchase'%name])/df_temp['%s_purchase_counts'%name]
        df_temp['base_diff_%s_purchase'%name] = df_temp['%s_month1_purchase'%name] - df_temp['%s_purchase_mean'%name]
    else:
        df_temp['%s_purchase_mean'%name] = (df_temp['%s_purchase_mean'%name] - df_temp['%s_month0_purchase'%name])/df_temp['%s_purchase_counts'%name]
        df_temp['base_diff_%s_purchase'%name] = df_temp['%s_month0_purchase'%name] - df_temp['%s_purchase_mean'%name]
    
    df_temp['base_diff_%s_purchaseRatio'%name] = df_temp['base_diff_%s_purchase'%name]/df_temp['%s_purchase_mean'%name]
    df_data = df_data.merge(df_temp,on='merchant_id',how='left')
    
    del df_purchase,df_temp,cols,df_count
    gc.collect()
    
    return df_data

#商家不同阶段的回购率1
def getMerchantRepurchaseRatio1(df_merchant,df_feature,group=['merchant_id','card_id'],fea='purchase_amount',name=''):
    #回购率
    df_temp = df_feature.groupby(group)['purchase_amount'].count().reset_index()
    df_temp.rename(columns={fea:'repurchase_count'},inplace=True)
    total = df_temp.shape[0]
    df_temp['repurchase_flag'] = (df_temp.repurchase_count>1)+0
    
    df_temp = df_temp.groupby('merchant_id')['repurchase_flag'].sum().reset_index()
    df_temp.rename(columns={'repurchase_flag':'repurchase_sum'},inplace=True)
    df_temp['lag_%s_repurchase_ratio_1'%name] = df_temp['repurchase_sum']/total
    
    df_temp.drop(columns=['repurchase_sum'],inplace=True)
    df_merchant = df_merchant.merge(df_temp,on='merchant_id',how='left')
    return df_merchant
#商家不同阶段的回购率2
def getMerchantRepurchaseRatio2(df_merchant,df_feature,group=['merchant_id','card_id'],fea='purchase_amount',name=''):
    df_temp = df_feature.groupby(group)['purchase_amount'].count().reset_index()
    df_temp.rename(columns={fea:'repurchase_count'},inplace=True)
    total = df_temp.shape[0]
    df_temp['repurchase_flag'] = (df_temp.repurchase_count>1)+0
    df_temp['repurchase_flag'] = df_temp['repurchase_flag']*df_temp['repurchase_count']
    
    df_temp = df_temp.groupby('merchant_id')['repurchase_flag'].sum().reset_index()
    df_temp.rename(columns={'repurchase_flag':'repurchase_sum'},inplace=True)
    df_temp['lag_%s_repurchase_ratio_2'%name] = df_temp['repurchase_sum']/total
    
    df_temp.drop(columns=['repurchase_sum'],inplace=True)
    df_merchant = df_merchant.merge(df_temp,on='merchant_id',how='left')
    return df_merchant

df_merchants = getMonthSales(df_merchants,df_historical_transactions,name='histMerch')
df_merchants = getMonthSales(df_merchants,df_new_transactions,name='newMerch')

for index in [-3,-6,-12]:
    df_feature = df_historical_transactions[df_historical_transactions.month_gap>index]
    df_merchants = getMerchantRepurchaseRatio1(df_merchants,df_feature,name='%s'%(-1*index))
    df_merchants = getMerchantRepurchaseRatio2(df_merchants,df_feature,name='%s'%(-1*index))
    
for index in [1,2]:
    df_feature = df_new_transactions[df_new_transactions.month_gap==index]
    df_merchants = getMerchantRepurchaseRatio1(df_merchants,df_feature,name='new%s'%index)
    df_merchants = getMerchantRepurchaseRatio2(df_merchants,df_feature,name='new%s'%index)

df_merchants.fillna(0,inplace=True)
del df_feature
gc.collect()

....................hist....................
....................new....................


105

#### merchant features

In [11]:
df_merchants.head()

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,lag_3_repurchase_ratio_1,lag_3_repurchase_ratio_2,lag_6_repurchase_ratio_1,lag_6_repurchase_ratio_2,lag_12_repurchase_ratio_1,lag_12_repurchase_ratio_2,lag_new1_repurchase_ratio_1,lag_new1_repurchase_ratio_2,lag_new2_repurchase_ratio_1,lag_new2_repurchase_ratio_2
0,M_ID_838061e48c,8353,792,9,-0.057465,-0.057465,N,E,E,-0.400000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M_ID_9339d880ad,3184,840,20,-0.057465,-0.057465,N,E,E,-0.720000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M_ID_e726bbae1e,447,690,1,-0.057465,-0.057465,N,E,E,-82.129997,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057465,-0.057465,Y,E,E,13.840252,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M_ID_64456c37ce,2228,222,21,-0.057465,-0.057465,Y,E,E,13.840252,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df_transactions = pd.read_csv('./datasets/df_transactions.csv',usecols=['card_id','merchant_id'])
df_uid = pd.read_csv('./datasets/df_data.csv',usecols=['card_id'])

In [14]:
%%time
idCols = ['merchant_id','merchant_group_id','merchant_category_id','subsector_id','category_2']
numMerchCols = [_f for _f in df_merchants.columns if _f not in idCols and df_merchants[_f].dtype!='object']
cateMerchCols = ['most_recent_sales_range','most_recent_purchases_range','category_4']
df_merchants = reduce_mem_usage(df_merchants)

df_merchants = label_encoding(df_merchants,cateMerchCols)

df_transactions = df_transactions.merge(df_merchants,on='merchant_id',how='left')

for fea in numMerchCols:
    df_uid = getMeanStaticsFeatures(df_uid,df_transactions,['card_id'],fea,name='%s_MeanMerchant'%fea)
    df_uid = getMaxStaticsFeatures(df_uid,df_transactions,['card_id'],fea,name='%s_MaxMerchant'%fea)
    df_uid = getStdStaticsFeatures(df_uid,df_transactions,['card_id'],fea,name='%s_StdMerchant'%fea)
    df_uid = getMedianStaticsFeatures(df_uid,df_transactions,['card_id'],fea,name='%s_MedianMerchant'%fea)
    
for fea in cateMerchCols:
    df_uid = getCategoryFrequenceMax(df_uid,df_transactions,['card_id'],fea,name='%s_frequenceMax'%fea)
    df_uid = getCategoryCounts(df_uid,df_transactions,['card_id'],fea,name='%s_categoryCounts'%fea)
    df_uid = getCategoryFrequenceMaxRatio(df_uid,df_transactions,['card_id'],fea,name='%s_frequenceMaxRatio'%fea)
    df_uid = getCategoryCountsRatio(df_uid,df_transactions,['card_id'],fea,name='%s_categoryCountsRatio'%fea)

# del df_merchants
gc.collect()
df_uid = reduce_mem_usage(df_uid)
df_uid.fillna(0,inplace=True)
df_uid.to_csv('./datasets/df_card_merchant_features.csv',index=False)

Memory usage of dataframe is 224.99 MB
Memory usage after optimization is: 99.25 MB
Decreased by 55.9%



Memory usage of dataframe is 379.69 MB
Memory usage after optimization is: 328.16 MB
Decreased by 13.6%
CPU times: user 1h 10min 26s, sys: 6min 44s, total: 1h 17min 11s
Wall time: 49min


In [16]:
df_uid.shape

(325540, 369)

#### region features

In [ ]:
def getRegionMerchantsCount(df_merchants,group=None,fea='',name=''):
    df_temp = df_merchants.groupby(group)[fea].count().reset_index()
    df_temp.rename(columns={fea:name+'_count'},inplace=True)
    df_temp['%s_ratio'%name] = df_temp[name+'_count']/df_merchants.shape[0]
    df_merchants = df_merchants.merge(df_temp,on=group,how='left')
    return df_merchants
def getRegionMerchantsUnique(df_merchants,group=None,fea='',name=''):
    df_temp = df_merchants.groupby(group)[fea].apply(lambda series:len(series.unique())).reset_index()
    df_temp.rename(columns={fea:name+'_unique'},inplace=True)
    df_temp['%s_ratio'%name] = df_temp[name+'_unique']/len(df_merchants[fea].unique())
    df_merchants = df_merchants.merge(df_temp,on=group,how='left')
    return df_merchants
#不同区域的商家占比、不同区域的商家类别占比
df_merchants = getRegionMerchantsCount(df_merchants,group=['subsector_id'],fea='merchant_id',name='subsector_merchant')
df_merchants = getRegionMerchantsUnique(df_merchants,group=['subsector_id'],fea='merchant_category_id',name='subsector_merchant_category')
#不同C2的商家占比、不同C2的商家类别占比
df_merchants = getRegionMerchantsCount(df_merchants,group=['category_2'],fea='merchant_id',name='C2_merchant')
df_merchants = getRegionMerchantsUnique(df_merchants,group='category_2',fea='merchant_category_id',name='C2_merchant_category')
#不同区域不同类别商家占比
df_merchants = getRegionMerchantsUnique(df_merchants,group=['subsector_id','merchant_category_id'],fea='merchant_id',name='subsector_MC')
#不同C2不同类别商家占比
df_merchants = getRegionMerchantsUnique(df_merchants,group=['category_2','merchant_category_id'],fea='merchant_id',name='C2_MC')

#不同区域的商业圈占比
df_merchants = getRegionMerchantsUnique(df_merchants,group=['subsector_id'],fea='merchant_group_id',name='subsector_merchant_group')
df_merchants = getRegionMerchantsUnique(df_merchants,group=['category_2'],fea='merchant_group_id',name='C2_merchant_group')
df_merchants = getRegionMerchantsCount(df_merchants,group=['category_2','merchant_group_id'],fea='merchant_id',name='C2_group_merchant')
df_merchants = getRegionMerchantsCount(df_merchants,group=['subsector_id','merchant_group_id'],fea='merchant_id',name='subsector_group_merchant')

#保存商家特征
df_merchants = df_merchants.drop(columns=['merchant_category_id','subsector_id','category_1','city_id','state_id','category_2'])
df_merchants.to_csv('./datasets/df_merchants.csv',index=False)

In [ ]:
df_uid = pd.read_csv('./datasets/train.csv',usecols=['card_id'])
df_transactions = pd.read_csv('./datasets/df_transactions.csv',usecols=['card_id','merchant_id','month_lag'])
df_transactions['card_merchant_id'] = df_transactions['card_id'] + '_' + df_transactions['merchant_id']

df_transactions['card_merchant_id'] = label_encoding(df_transactions,['card_merchant_id'])

In [ ]:
df_historical_transactions = df_transactions[df_transactions.month_lag<=0][['card_id','card_merchant_id']]
df_new_transactions = df_transactions[df_transactions.month_lag>0][['card_id','card_merchant_id']]

del df_transactions
gc.collect()

print('..........do something..............')
for flag,df_feature in zip(['hist','new'],[df_historical_transactions,df_new_transactions]):
#     df_uid = getCategoryCounts(df_uid,df_feature,['card_id'],'card_merchant_id',name='%s_%s_categoryCounts'%(flag,'(card,merchant)'))
#     df_uid = getCategoryFrequenceMaxRatio(df_uid,df_feature,['card_id'],'card_merchant_id',name='%s_%s_frequenceMaxRatio'%(flag,'(card,merchant)'))
    df_uid = getCategoryCountsRatio(df_uid,df_feature,['card_id'],'card_merchant_id',name='%s_%s_categoryCountsRatio'%(flag,'(card,merchant)'))
    
df_uid.to_csv('./datasets/df_card_merchant_features.csv',index=False)
del df_new_transactions,df_historical_transactions
gc.collect()